In [1]:
#import all required/helpful toolboxes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  
%matplotlib inline
import hddm
print hddm.__version__
from patsy import dmatrix


0.6.0


C:\Users\mrowl.WDOCN\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [3]:
#load data
data_all = hddm.load_csv('D:\Neural Stability Pilot\GitHub\data\Study2\Data_HDDM.csv') #define file and directory
data_all.head(10)


,response,rt,Post_Coherence,Pre_Coherence,Accuracy_initial,World_state,Decision_Initial,Confidence_Initial,subj_idx
0,1,0.89550,1,2,1,-1,1,0.332,1
1,1,0.72935,1,1,1,1,-1,-0.332,1
2,1,0.76379,2,2,1,-1,1,-0.668,1
3,1,0.82256,2,1,1,-1,1,-0.332,1
4,0,0.79702,2,2,-1,1,1,-0.332,1
5,1,0.79717,1,2,1,-1,1,-0.332,1
6,1,1.36330,1,1,-1,1,1,-0.332,1
7,0,0.79685,2,2,-1,-1,-1,0.332,1
8,1,0.73012,2,2,1,1,-1,-0.332,1
9,1,0.76343,1,2,1,-1,1,0.000,1


In [4]:
# define link function for the starting-point
def z_link_func(x, data=data_all):
    stim=(np.transpose(np.asarray(data.World_state))) 

    return 1 / (1 + np.exp(-(x)))

# define regressions for all the different models

# Model 1: baseline
z_reg1 = {'model': 'z ~ 1 ', 'link_func': z_link_func}
v_reg1 = {'model': 'v ~ 1 + Post_Coherence', 'link_func': lambda x: x}
a_reg1 = {'model': 'a ~ 1 + Confidence_Initial', 'link_func': lambda x: x}

reg_descr1 = [z_reg1, v_reg1, a_reg1]

# Model 2-4: dependencies on confidence
z_reg2 = {'model': 'z ~ 1+Confidence_Initial', 'link_func': z_link_func}
v_reg2 = {'model': 'v ~ 1 + Post_Coherence', 'link_func': lambda x: x}
reg_descr2 = [z_reg2, v_reg2, a_reg1]


z_reg3 = {'model': 'z ~ 1', 'link_func': z_link_func}
v_reg3 = {'model': 'v ~ 1 + Post_Coherence+ Confidence_Initial', 'link_func': lambda x: x}
reg_descr3 = [z_reg3, v_reg3, a_reg1]


z_reg4 = {'model': 'z ~ 1+ Confidence_Initial', 'link_func': z_link_func}
v_reg4 = {'model': 'v ~ 1 + Post_Coherence+ Confidence_Initial', 'link_func': lambda x: x}
reg_descr4 = [z_reg4, v_reg4, a_reg1]

# Model 5-7: dependencies on initial decisions
z_reg5 = {'model': 'z ~ 1+Accuracy_initial', 'link_func': z_link_func}
v_reg5 = {'model': 'v ~ 1 + Post_Coherence', 'link_func': lambda x: x}
reg_descr5 = [z_reg5, v_reg5, a_reg1]


z_reg6 = {'model': 'z ~ 1', 'link_func': z_link_func}
v_reg6= {'model': 'v ~ 1 + Post_Coherence+ Accuracy_initial', 'link_func': lambda x: x}
reg_descr6 = [z_reg6, v_reg6, a_reg1]


z_reg7 = {'model': 'z ~ 1+ Accuracy_initial', 'link_func': z_link_func}
v_reg7 = {'model': 'v ~ 1 + Post_Coherence+ Accuracy_initial', 'link_func': lambda x: x}
reg_descr7 = [z_reg7, v_reg7, a_reg1]


# Model 8-10: dependencies on interaction
z_reg8 = {'model': 'z ~ 1+ Confidence_Initial+Accuracy_initial+Confidence_Initial:Accuracy_initial', 'link_func': z_link_func}
v_reg8 = {'model': 'v ~ 1 + Post_Coherence', 'link_func': lambda x: x}
reg_descr8 = [z_reg8, v_reg8, a_reg1]

z_reg9 = {'model': 'z ~ 1', 'link_func': z_link_func}
v_reg9 = {'model': 'v ~ 1 + Post_Coherence+ Confidence_Initial+Accuracy_initial+Confidence_Initial:Accuracy_initial', 'link_func': lambda x: x}
reg_descr9 = [z_reg9, v_reg9, a_reg1]

z_reg10 = {'model': 'z ~ 1+ Confidence_Initial+Accuracy_initial+Confidence_Initial:Accuracy_initial', 'link_func': z_link_func}
v_reg10 = {'model': 'v ~ 1 + Post_Coherence+ Confidence_Initial+Accuracy_initial+Confidence_Initial:Accuracy_initial', 'link_func': lambda x: x}
reg_descr10 = [z_reg10, v_reg10, a_reg1]

In [ ]:
#fit the models

m_within_subj_all_1 = hddm.HDDMRegressor(data_all,  reg_descr1, include='z',  p_outlier=0.05)
m_within_subj_all_1.find_starting_values()
m_within_subj_all_1.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_1.save('Model1')

m_within_subj_all_2 = hddm.HDDMRegressor(data_all,  reg_descr2, include='z',  p_outlier=0.05)
m_within_subj_all_2.find_starting_values()
m_within_subj_all_2.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_2.save('Model2')

m_within_subj_all_3 = hddm.HDDMRegressor(data_all,  reg_descr3, include='z',  p_outlier=0.05)
m_within_subj_all_3.find_starting_values()
m_within_subj_all_3.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_3.save('Model3')


m_within_subj_all_4 = hddm.HDDMRegressor(data_all,  reg_descr4, include='z',  p_outlier=0.05)
m_within_subj_all_4.find_starting_values()
m_within_subj_all_4.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_4.save('Model4')

m_within_subj_all_5 = hddm.HDDMRegressor(data_all,  reg_descr5, include='z',  p_outlier=0.05)
m_within_subj_all_5.find_starting_values()
m_within_subj_all_5.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_5.save('Model5')


m_within_subj_all_6 = hddm.HDDMRegressor(data_all,  reg_descr6, include='z',  p_outlier=0.05)
m_within_subj_all_6.find_starting_values()
m_within_subj_all_6.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_6.save('Model6')


m_within_subj_all_7 = hddm.HDDMRegressor(data_all,  reg_descr7, include='z',  p_outlier=0.05)
m_within_subj_all_7.find_starting_values()
m_within_subj_all_7.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_7.save('Model7')


m_within_subj_all_8 = hddm.HDDMRegressor(data_all,  reg_descr8, include='z',  p_outlier=0.05)
m_within_subj_all_8.find_starting_values()
m_within_subj_all_8.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_8.save('Model8')

m_within_subj_all_9 = hddm.HDDMRegressor(data_all,  reg_descr9, include='z',  p_outlier=0.05)
m_within_subj_all_9.find_starting_values()
m_within_subj_all_9.sample(100000, burn=75000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_9.save('Model9')

m_within_subj_all_10 = hddm.HDDMRegressor(data_all,  reg_descr10, include='z',  p_outlier=0.05)
m_within_subj_all_10.find_starting_values()
m_within_subj_all_10.sample(100000, burn=50000, thin=25, dbname='traces.db', db='pickle')
m_within_subj_all_10.save('Model10')

Adding these covariates:
['z_Intercept', 'z_Confidence_Initial', 'z_Accuracy_initial', 'z_Confidence_Initial:Accuracy_initial']
Adding these covariates:
['v_Intercept', 'v_Post_Coherence']


C:\Users\mrowl.WDOCN\AppData\Local\Continuum\Anaconda2\lib\site-packages\scipy\optimize\optimize.py:1850: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
C:\Users\mrowl.WDOCN\AppData\Local\Continuum\Anaconda2\lib\site-packages\scipy\optimize\optimize.py:1855: RuntimeWarning: invalid value encountered in absolute
  tmp2 = numpy.abs(tmp2)
C:\Users\mrowl.WDOCN\AppData\Local\Continuum\Anaconda2\lib\site-packages\scipy\optimize\optimize.py:1851: RuntimeWarning: invalid value encountered in double_scalars
  p = (x - v) * tmp2 - (x - w) * tmp1


In [ ]:
m_within_subj_all_1.print_stats()
m_within_subj_all_2.print_stats()
m_within_subj_all_3.print_stats()
m_within_subj_all_4.print_stats()
m_within_subj_all_5.print_stats()
m_within_subj_all_6.print_stats()
m_within_subj_all_7.print_stats()
m_within_subj_all_8.print_stats()
m_within_subj_all_9.print_stats()
m_within_subj_all_10.print_stats()

In [13]:
# save data of the winning model to csv
stats_10= m_within_subj_all_10.gen_stats()
df10 = pd.DataFrame(data=stats_10)
path_d = 'group_hddm_10.csv'
df10.to_csv(path_or_buf=path_d)

Adding these covariates:
['z_Intercept', 'z_Confidence_Initial', 'z_Accuracy_initial', 'z_Confidence_Initial:Accuracy_initial']
Adding these covariates:
['v_Intercept', 'v_Post_Coherence']
 [-----------------100%-----------------] 100001 of 100000 complete in 140890.8 secWARNING: Will not save custom link functions.
Adding these covariates:
['z_Intercept']
Adding these covariates:
['v_Intercept', 'v_Post_Coherence', 'v_Confidence_Initial', 'v_Accuracy_initial', 'v_Confidence_Initial:Accuracy_initial']
 [-----------------100%-----------------] 100001 of 100000 complete in 128830.4 secWARNING: Will not save custom link functions.


In [ ]:
# plot posterior parameter distributions

# define the relevant parameter distributions
starting_point_M_conf = m_within_subj_all_10.nodes_db.node['z_Confidence_Initial']
drift_rate_M_conf = m_within_subj_all_10.nodes_db.node['v_Confidence_Initial']

starting_point_M_consistency = m_within_subj_all_10.nodes_db.node['z_Accuracy_initial']
drift_rate_M_consistency = m_within_subj_all_10.nodes_db.node['v_Accuracy_initial']

starting_point_interaction = m_within_subj_all_10.nodes_db.node['z_Confidence_Initial:Accuracy_initial']
drift_rate_interaction = m_within_subj_all_10.nodes_db.node['v_Confidence_Initial:Accuracy_initial']


# Plot Figure 2D
plot_1=hddm.analyze.plot_posterior_nodes([starting_point_M_conf, drift_rate_M_conf], bins=14)
plt.legend('')
plt.xlim(left=-.35, right=.4)
plt.savefig('D:\Neural Stability Pilot\Figures\Figure_2D_left.pdf', format='pdf', dpi=1000)

plot_2=hddm.analyze.plot_posterior_nodes([starting_point_M_consistency, drift_rate_M_consistency], bins=40)
plt.legend('')
plt.xlim(left=-.03, right=.6)
plt.savefig('D:\Neural Stability Pilot\Figures\Figure_2D_middle.pdf', format='pdf', dpi=1000)


plot_3=hddm.analyze.plot_posterior_nodes([starting_point_interaction, drift_rate_interaction], bins=12)
plt.legend('')
plt.xlim(left=-.03, right=.62)
plt.savefig('D:\Neural Stability Pilot\Figures\Figure_2D_right.pdf', format='pdf', dpi=1000)
